## Задача 3.1
Выборка : https://github.com/natasha/nerus набор предложений на русском языке с указанием частей речи для каждого слова.

Требуется:
1. Рассмотреть последовательность частей речи как марковскую модель. Определить оптимальный порядок марковской модели.
2. Обучить скрытую марковскую модель по выборке. Оценить точность предсказания частей речи, посчитать энтропию на выборке.

*Важно:* в целях ускорения эксперимента рекомендуется взять первые 10 МБ текста из выборки.

In [4]:
import re

import urllib
import numpy as np

import nltk
from nltk.tag import hmm
import pandas as pd

from collections import Counter

import nltk.lm as lm
from nltk.util import ngrams as nltk_ngrams
import numpy as np
import scipy.stats as st

from nltk.tokenize import RegexpTokenizer

from nerus import load_nerus
import sys
from typing import List, Tuple

In [5]:
# !wget https://storage.yandexcloud.net/natasha-nerus/data/nerus_lenta.conllu.gz -O nerus_lenta.conllu.gz -q

### Загрузка данных

In [6]:
docs = load_nerus('nerus_lenta.conllu.gz')

In [7]:
# list of pairs: (token, tag)
dataset: List[Tuple[str, str]] = []
size = 0
for i, doc in enumerate(docs):
    for sent in doc.sents:
        tokens = sent.morph.tokens
        texts = [t.text.lower() for t in tokens]
        tags = [t.pos for t in tokens]
        size += sys.getsizeof(texts) + sys.getsizeof(tags)
        dataset.extend([(text, tag) for text, tag in zip(texts, tags)])
    if size > 10 * (1024) ** 2:
        break

In [8]:
dataset[:10]

[('вице-премьер', 'NOUN'),
 ('по', 'ADP'),
 ('социальным', 'ADJ'),
 ('вопросам', 'NOUN'),
 ('татьяна', 'PROPN'),
 ('голикова', 'PROPN'),
 ('рассказала', 'VERB'),
 (',', 'PUNCT'),
 ('в', 'ADP'),
 ('каких', 'DET')]

In [9]:
# vocabulary of russian words
vocab_russian = set([el[0] for el in dataset])
vocab_tags = set([el[1] for el in dataset])

### Определение оптимального порядка скрытой марковской модели

Рассмотрим последовательность тэгов. Проверим гипотезу о том, что марковскую цепь второго порядка можно свернуть в цепь первого порядка. Воспользуемся критерием с лекции и соответствующим кодом с семинара.


In [10]:
tags = [el[1] for el in dataset]

In [12]:
tags[:10]

['NOUN', 'ADP', 'ADJ', 'NOUN', 'PROPN', 'PROPN', 'VERB', 'PUNCT', 'ADP', 'DET']

In [13]:
# seminar
def ngrams_and_prefix_counts(tokens, n_max):
    # словарь n-грамм и их частот
    ngrams_counts = {}
    # словарь n-граммных префиксов и их частот
    prefix_counts = {}
    
    n = len(tokens)
    for i in range(n_max):
        ngrams_counts[i + 1] = Counter([tuple(tokens[j : j + i + 1]) for j in range(n - i)])
        prefix_counts[i + 1] = Counter([tuple(tokens[j : j + i] + ['*']) for j in range(n - i)])

    return ngrams_counts, prefix_counts


def unigram_probas(ngram_counts):
    p1 = {}
    n = sum(ngram_counts[1].values())
    for w in ngram_counts[1]:
        p1[w] = ngram_counts[1][w] / n
    return p1


def bigram_probas(ngram_counts, prefix_counts):
    p2 = {}
    for w in ngram_counts[2]:
        pre_w = tuple([w[0]] + ['*'])
        p2[u'{1}|{0}'.format(*w)] = ngram_counts[2][w] / prefix_counts[2][pre_w]
    return p2


def trigram_probas(ngram_counts, prefix_counts):
    p3 = {}
    for w in ngram_counts[3]:
        pre_w = w[:2] + tuple(['*'])
        p3[u'{2}|{1},{0}'.format(*w)] = ngram_counts[3][w] / prefix_counts[3][pre_w]
    return p3


In [14]:
ngram_counts, prefix_counts = ngrams_and_prefix_counts(tags, 3)

In [15]:
p1 = unigram_probas(ngram_counts)
p2 = bigram_probas(ngram_counts, prefix_counts)
p3 =  trigram_probas(ngram_counts, prefix_counts)

In [16]:
def chi2_statistic(p2, p3, tokens):
    stat2 = []
    stat3 = []
    n = len(tokens)
    for i in range(n - 2):
        w = tokens[i : i + 3]
        ngram3 = '{2}|{1},{0}'.format(*w)
        ngram2 = '{1}|{0}'.format(*w)

        stat2.append(np.log(p2[ngram2]))
        stat3.append(np.log(p3[ngram3]))
    return - 2 * np.sum(stat2) + 2 * np.sum(stat3)

In [17]:
m = len(p3)
stat = chi2_statistic(p2, p3, tags)

In [18]:
print(f'p-value = {1 - st.distributions.chi2(m * ((m - 1) ** 2) - 1).cdf(stat)}')

p-value = 1.0


$\mathrm{p\_value} > 0.05$. Значит, данные гипотезе не противоречат. Оптимальная цепь имеет порядок 1.

### Обучение HMM оптимального порядка

In [24]:
np.random.seed(0)
trainer = hmm.HiddenMarkovModelTrainer(states=vocab_tags, symbols=vocab_russian)

In [25]:
# train/val split
val_samples = int(0.1 * len(dataset))
train_dataset = dataset[:-val_samples]
val_dataset = dataset[-val_samples:]

In [26]:
%%time
tagger = trainer.train_supervised([train_dataset])

CPU times: user 638 ms, sys: 8.62 ms, total: 646 ms
Wall time: 658 ms


### Точность и энтропия на данных из обучающей выборки

* Accuracy: 98.20
* Entropy: 22.5

In [27]:
result = tagger.test([train_dataset[:500]], verbose=True)

Test: вице-премьер/NOUN по/ADP социальным/ADJ вопросам/NOUN татьяна/PROPN голикова/PROPN рассказала/VERB ,/PUNCT в/ADP каких/DET регионах/NOUN россии/PROPN зафиксирована/VERB наиболее/ADV высокая/ADJ смертность/NOUN от/ADP рака/NOUN ,/PUNCT сообщает/VERB риа/PROPN новости/PROPN ./PUNCT по/ADP словам/NOUN голиковой/PROPN ,/PUNCT чаще/ADV всего/PRON онкологические/ADJ заболевания/NOUN становились/VERB причиной/NOUN смерти/NOUN в/ADP псковской/ADJ ,/PUNCT тверской/ADJ ,/PUNCT тульской/ADJ и/CCONJ орловской/ADJ областях/NOUN ,/PUNCT а/CCONJ также/ADV в/ADP севастополе/PROPN ./PUNCT вице-премьер/NOUN напомнила/VERB ,/PUNCT что/SCONJ главные/ADJ факторы/NOUN смертности/NOUN в/ADP россии/PROPN —/PUNCT рак/NOUN и/CCONJ болезни/NOUN системы/NOUN кровообращения/NOUN ./PUNCT в/ADP начале/NOUN года/NOUN стало/VERB известно/ADJ ,/PUNCT что/SCONJ смертность/NOUN от/ADP онкологических/ADJ заболеваний/NOUN среди/ADP россиян/NOUN снизилась/VERB впервые/ADV за/ADP три/NUM года/NOUN ./PUNCT по/ADP данным

### Точность и энтропия на данных из валидационной выборки

* Accuracy: 0.0
* Entropy: $\infty \Rightarrow$ модель не описывает выборку

In [29]:
result = tagger.test([val_dataset[:500]], verbose=True)

Test: детской/ADJ одежды/NOUN сайт/NOUN распространял/VERB информацию/NOUN о/ADP сексуальных/ADJ услугах/NOUN с/ADP участием/NOUN несовершеннолетних/NOUN ,/PUNCT содержал/VERB порнографические/ADJ описания/NOUN детей/NOUN 5-8/NUM лет/NOUN ./PUNCT ранее/ADV провести/VERB проверку/NOUN по/ADP сообщениям/NOUN сми/NOUN пообещали/VERB в/ADP столичном/ADJ управлении/NOUN следственного/ADJ комитета/NOUN россии/PROPN ./PUNCT премьер-министр/NOUN франции/PROPN эдуар/PROPN филипп/PROPN объявил/VERB ,/PUNCT что/SCONJ правительство/NOUN вводит/VERB мораторий/NOUN на/ADP повышение/NOUN тарифов/NOUN на/ADP топливо/NOUN сроком/NOUN на/ADP полгода/NOUN ./PUNCT об/ADP этом/PRON сообщает/VERB агентство/NOUN reuters/PROPN ./PUNCT планировавшийся/VERB рост/NOUN тарифов/NOUN будет/AUX заморожен/VERB с/ADP 1/ADJ января/NOUN 2019/ADJ года/NOUN ./PUNCT такое/DET решение/NOUN было/AUX принято/VERB из-за/ADP массовых/ADJ протестов/NOUN ./PUNCT «/PUNCT этот/DET гнев/NOUN —/PUNCT нужно/ADJ быть/AUX глухим/ADJ или